In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('bdr20-21data.txt')
winter2 = pd.read_csv ('bdr21-22data.txt')
winter3 = pd.read_csv ('bdr22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 52]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 52]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 52]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
bdr = LocationInfo(
    'Bridgeport', 'Connecticut', 
    latitude=41.1792,  
    longitude=-73.1894,
    timezone='America/Connecticut'
)

bdr

LocationInfo(name='Bridgeport', region='Connecticut', timezone='America/Connecticut', latitude=41.1792, longitude=-73.1894)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(bdr.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(bdr.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(bdr.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:59:33  2020-12-01 16:24:16  2020-12-01
1   2020-12-02 07:00:34  2020-12-02 16:24:02  2020-12-02
2   2020-12-03 07:01:34  2020-12-03 16:23:51  2020-12-03
3   2020-12-04 07:02:33  2020-12-04 16:23:42  2020-12-04
4   2020-12-05 07:03:30  2020-12-05 16:23:35  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:34:30  2021-02-24 17:37:52  2021-02-24
86  2021-02-25 06:33:00  2021-02-25 17:39:03  2021-02-25
87  2021-02-26 06:31:30  2021-02-26 17:40:13  2021-02-26
88  2021-02-27 06:29:58  2021-02-27 17:41:24  2021-02-27
89  2021-02-28 06:28:26  2021-02-28 17:42:33  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:18  2021-12-01 16:24:19  2021-12-01
1   2021-12-02 07:00:20  2021-12-02 16:24:05  2021-12-02
2   2021-12-03 07:01:20  2021-12-03 16:23:53  2021-12-03
3   2021-12-04 07:02:19  2021-12-04 16:23:44  2021-12-04
4   2021-12-05 07:03:16  2021-12-05 16:23:36  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:34:52  2022-02-24 17:37:35  2022-02-24
86  2022-02-25 06:33:22  2022-02-25 17:38:46  2022-02-25
87  2022-02-26 06:31:52  2022-02-26 17:39:56  2022-02-26
88  2022-02-27 06:30:20  2022-02-27 17:41:06  2022-02-27
89  2022-02-28 06:28:48  2022-02-28 17:42:16  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:03  2022-12-01 16:24:23  2022-12-01
1   2022-12-02 07:00:05  2022-12-02 16:24:08  2022-12-02
2   2022-12-03 07:01:05  2022-12-03 16:23:56  2022-12-03
3   2022-12-04 07:02:05  2022-12-04 16:23:45  2022-12-04
4   2022-12-05 07:03:03  2022-12-05 16:23:38  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:35:13  2023-02-24 17:37:18  2023-02-24
86  2023-02-25 06:33:44  2023-02-25 17:38:28  2023-02-25
87  2023-02-26 06:32:14  2023-02-26 17:39:39  2023-02-26
88  2023-02-27 06:30:43  2023-02-27 17:40:49  2023-02-27
89  2023-02-28 06:29:11  2023-02-28 17:42:00  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDR,2020-12-14 19:52:00,0.0000,-SN,2020-12-14,2020-12-14 07:11:03,2020-12-14 16:24:20
1,BDR,2020-12-16 22:52:00,0.0001,-SN,2020-12-16,2020-12-16 07:12:25,2020-12-16 16:24:55
2,BDR,2020-12-17 00:52:00,1.2700,SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
3,BDR,2020-12-17 01:52:00,2.0300,+SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
4,BDR,2020-12-17 02:52:00,2.2900,-SN BR,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
...,...,...,...,...,...,...,...
81,BDR,2021-02-19 17:52:00,0.5100,-SN,2021-02-19,2021-02-19 06:41:45,2021-02-19 17:31:54
82,BDR,2021-02-19 21:52:00,0.0001,-SN,2021-02-19,2021-02-19 06:41:45,2021-02-19 17:31:54
83,BDR,2021-02-20 00:52:00,0.5100,-SN BR,2021-02-20,2021-02-20 06:40:20,2021-02-20 17:33:06
84,BDR,2021-02-20 23:52:00,0.0000,-SN,2021-02-20,2021-02-20 06:40:20,2021-02-20 17:33:06


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDR,2021-12-08 17:52:00,0.5100,-SN,2021-12-08,2021-12-08 07:06:02,2021-12-08 16:23:29
1,BDR,2021-12-08 18:52:00,0.2500,-SN,2021-12-08,2021-12-08 07:06:02,2021-12-08 16:23:29
2,BDR,2021-12-09 00:52:00,0.0001,-SN BR,2021-12-09,2021-12-09 07:06:54,2021-12-09 16:23:31
3,BDR,2021-12-09 01:52:00,0.2500,-SN,2021-12-09,2021-12-09 07:06:54,2021-12-09 16:23:31
4,BDR,2021-12-09 02:52:00,0.2500,-SN BR,2021-12-09,2021-12-09 07:06:54,2021-12-09 16:23:31
...,...,...,...,...,...,...,...
69,BDR,2022-02-13 23:52:00,0.0001,-SN,2022-02-13,2022-02-13 06:50:11,2022-02-13 17:24:16
70,BDR,2022-02-19 17:52:00,0.0001,-SN,2022-02-19,2022-02-19 06:42:06,2022-02-19 17:31:36
71,BDR,2022-02-19 18:52:00,0.0001,-SN,2022-02-19,2022-02-19 06:42:06,2022-02-19 17:31:36
72,BDR,2022-02-19 20:52:00,0.0001,+SN,2022-02-19,2022-02-19 06:42:06,2022-02-19 17:31:36


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDR,2022-12-11 17:52:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
1,BDR,2022-12-11 18:52:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
2,BDR,2022-12-11 19:52:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
3,BDR,2022-12-11 20:52:00,0.0001,-SN,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
4,BDR,2022-12-11 21:52:00,0.5100,-SN BR,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
5,BDR,2022-12-11 22:52:00,0.2500,-SN,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
6,BDR,2022-12-11 23:52:00,0.5100,-SN BR,2022-12-11,2022-12-11 07:08:22,2022-12-11 16:23:40
7,BDR,2022-12-12 00:52:00,0.2500,-SN,2022-12-12,2022-12-12 07:09:10,2022-12-12 16:23:48
8,BDR,2022-12-12 01:52:00,0.0001,-SN,2022-12-12,2022-12-12 07:09:10,2022-12-12 16:23:48
9,BDR,2022-12-12 02:52:00,0.0001,-SN,2022-12-12,2022-12-12 07:09:10,2022-12-12 16:23:48


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
8,BDR,2020-12-17 08:52:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
17,BDR,2021-01-20 11:52:00,0.0001,-SN,2021-01-20,2021-01-20 07:13:27,2021-01-20 16:54:43
18,BDR,2021-01-20 12:52:00,0.0001,-SN,2021-01-20,2021-01-20 07:13:27,2021-01-20 16:54:43
19,BDR,2021-01-26 16:52:00,0.0001,-SN,2021-01-26,2021-01-26 07:09:09,2021-01-26 17:02:03
28,BDR,2021-01-27 07:52:00,0.2500,-SN BR,2021-01-27,2021-01-27 07:08:19,2021-01-27 17:03:18
29,BDR,2021-01-27 09:52:00,0.0001,-SN BR,2021-01-27,2021-01-27 07:08:19,2021-01-27 17:03:18
30,BDR,2021-01-29 09:52:00,0.0001,-SN,2021-01-29,2021-01-29 07:06:33,2021-01-29 17:05:48
33,BDR,2021-02-01 07:52:00,0.5100,SN FZFG,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34
34,BDR,2021-02-01 08:52:00,0.7600,-SN BR,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34
35,BDR,2021-02-01 09:52:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,BDR,2020-12-17 08:52:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16,2020-12-17 08:13:03,2020-12-17 15:25:16
17,BDR,2021-01-20 11:52:00,0.0001,-SN,2021-01-20,2021-01-20 07:13:27,2021-01-20 16:54:43,2021-01-20 08:13:27,2021-01-20 15:54:43
18,BDR,2021-01-20 12:52:00,0.0001,-SN,2021-01-20,2021-01-20 07:13:27,2021-01-20 16:54:43,2021-01-20 08:13:27,2021-01-20 15:54:43
29,BDR,2021-01-27 09:52:00,0.0001,-SN BR,2021-01-27,2021-01-27 07:08:19,2021-01-27 17:03:18,2021-01-27 08:08:19,2021-01-27 16:03:18
30,BDR,2021-01-29 09:52:00,0.0001,-SN,2021-01-29,2021-01-29 07:06:33,2021-01-29 17:05:48,2021-01-29 08:06:33,2021-01-29 16:05:48
34,BDR,2021-02-01 08:52:00,0.7600,-SN BR,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34,2021-02-01 08:03:42,2021-02-01 16:09:34
35,BDR,2021-02-01 09:52:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34,2021-02-01 08:03:42,2021-02-01 16:09:34
36,BDR,2021-02-01 10:52:00,1.0200,-SN BR,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34,2021-02-01 08:03:42,2021-02-01 16:09:34
37,BDR,2021-02-01 11:52:00,0.5100,-SN BR,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34,2021-02-01 08:03:42,2021-02-01 16:09:34
38,BDR,2021-02-01 13:52:00,0.2500,SN FZFG,2021-02-01,2021-02-01 07:03:42,2021-02-01 17:09:34,2021-02-01 08:03:42,2021-02-01 16:09:34


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.37079615384615383
2021-2022: 0.665186206896552
2022-2023: 0.09189090909090908


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BDR,2020-12-14 19:52:00,0.0000,-SN,2020-12-14,2020-12-14 07:11:03,2020-12-14 16:24:20
1,BDR,2020-12-16 22:52:00,0.0001,-SN,2020-12-16,2020-12-16 07:12:25,2020-12-16 16:24:55
2,BDR,2020-12-17 00:52:00,1.2700,SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
3,BDR,2020-12-17 01:52:00,2.0300,+SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
4,BDR,2020-12-17 02:52:00,2.2900,-SN BR,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
5,BDR,2020-12-17 03:52:00,0.5100,-SN BR,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
6,BDR,2020-12-17 04:52:00,4.8300,+SN FZFG,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
7,BDR,2020-12-17 05:52:00,1.2700,-SN BR,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
9,BDR,2020-12-17 16:52:00,0.2500,-SN,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16
10,BDR,2020-12-17 17:52:00,0.5100,-SN,2020-12-17,2020-12-17 07:13:03,2020-12-17 16:25:16


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_17696\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.9287130434782608
2021-2022: 0.27391538461538467
2022-2023: 0.1588125


In [65]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'CT'

data.to_csv('SO.bdr.csv', index=False)